In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import requests
from requests.adapters import HTTPAdapter, Retry
import os
import glob
import re
import warnings
warnings.simplefilter("ignore")



print('done')

done


In [81]:
def pullout_GO_info(df):
    df[['1','class','2']]= df['Category'].str.split('_', expand = True)
    df[['3','pathway']] = df['Term'].str.split('~', expand = True)
    df=df.rename(columns={'Count':'count','Fold Enrichment':'enrichment','PValue':'pvalue'})
    df['enrichment']=np.log2(df['enrichment'])
    df=df[['pathway','enrichment','pvalue','count','class']]
    df=df.loc[df['pvalue'] <= 0.05]
    return df
def pullout_KEGG(df):
    df[['class','1']]= df['Category'].str.split('_', expand = True)
    df[['2','pathway']] = df['Term'].str.split(':', expand = True)
    df=df.rename(columns={'Count':'count','Fold Enrichment':'enrichment','PValue':'pvalue'})
    df['enrichment']=np.log2(df['enrichment'])
    df=df[['pathway','enrichment','pvalue','count','class']]
    df=df.loc[df['pvalue'] <= 0.05]
    return df
def pullout_UP_PTM(df):
    df[['2','pathway']] = df['Term'].str.split('~', expand = True)-
    df=df.rename(columns={'Category':'class','Count':'count','Fold Enrichment':'enrichment','PValue':'pvalue'})
    df['enrichment']=np.log2(df['enrichment'])
    df=df[['pathway','enrichment','pvalue','count','class']]
    df=df.loc[df['pvalue'] <= 0.05]
    return df
def extract_data(directory):
    files = os.listdir(directory)
    txt_files = [file for file in files if file.endswith('.txt') or file.endswith('.csv')]

    # Read each txt file as an individual dataframe
    individual_dfs = {}
    for file in txt_files:
        df = pd.read_csv(os.path.join(directory, file), sep='\t')
        individual_dfs[file] = df
    
    data_dict = {}
    GO = 'GO'
    KEGG = 'KEGG'
    UP_PTM = 'PTM'
    GO_dfs = {key: df for key, df in individual_dfs.items() if GO in key}
    for key, df in GO_dfs.items():
        df_GO = pullout_GO_info(df)
        # Add the merged_df to the matching_dict
        data_dict[f'simple_{key}'] = df_GO
        
    KEGG_dfs = {key: df for key, df in individual_dfs.items() if KEGG in key}
    for key, df in KEGG_dfs.items():
        df_KEGG = pullout_KEGG(df)
        # Add the merged_df to the matching_dict
        data_dict[f'simple_{key}'] = df_KEGG
        
    UP_PTM_dfs = {key:df for key, df in individual_dfs.items() if UP_PTM in key}
    for key, df in UP_PTM_dfs.items():
        df_UP_PTM = pullout_UP_PTM(df)
        data_dict[f'simple_{key}'] = df_UP_PTM
    return data_dict

def save_dfs_to_csv(df_dict, save_directory):
    # Make sure the directory exists, if not, create it
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    for key, df in df_dict.items():
        # Specify the path of the excel file
        file_path = os.path.join(save_directory, f'{key}')
        
        # Save the dataframe to an excel file
        df.to_csv(file_path)
    
    

In [78]:
directory_of_DAVID = 'DAVID annotations/DAAM'
DAAM_dict = extract_data(directory_of_DAVID)
save_dfs_to_csv(DAAM_dict,'Output/DAAM')

In [84]:
directory_of_PCM1 = 'DAVID annotations/PCM-1'
PCM1_dict = extract_data(directory_of_PCM1)
save_dfs_to_csv(PCM1_dict,'Output/PCM-1')

In [85]:
directory_of_PCM2 = 'DAVID annotations/PCM-2'
PCM2_dict = extract_data(directory_of_PCM2)
save_dfs_to_csv(PCM2_dict,'Output/PCM-2')

In [ ]:
df_DAAM_BP = pd.read_csv('DAVID annotations/DAAM/DAAM_1p6_GO_BP.txt', sep='\t')
df_DAAM_CC = pd.read_csv('DAVID annotations/DAAM/DAAM_1p6_GO_CC.txt', sep='\t')
df_DAAM_MF = pd.read_csv('DAVID annotations/DAAM/DAAM_1p6_GO_MF.txt', sep='\t')
df_DAAM_KEGG = pd.read_csv('DAVID annotations/DAAM/DAAM_1p6_KEGG.txt', sep='\t')

df_DAAM_BP_simp = pullout_GO_info(df_DAAM_BP)
df_DAAM_CC_simp = pullout_GO_info(df_DAAM_CC)
df_DAAM_MF_simp = pullout_GO_info(df_DAAM_MF)
dfs_DAAM = [df_DAAM_BP_simp,df_DAAM_CC_simp,df_DAAM_MF_simp]
df_DAAM_Comb = pd.concat(dfs_DAAM)

df_DAAM_BP_simp.to_excel('Output/DAAM/DAAM_BP.xlsx')
df_DAAM_CC_simp.to_excel('Output/DAAM/DAAM_CC.xlsx')
df_DAAM_MF_simp.to_excel('Output/DAAM/DAAM_MF.xlsx')
df_DAAM_Comb.to_excel('Output/DAAM/DAAM_GO_combined.xlsx')


df_PCM1_BP = pd.read_csv('DAVID annotations/PCM-1/PCM-1_1p3_GO_BP.txt', sep='\t')
df_PCM1_CC = pd.read_csv('DAVID annotations/PCM-1/PCM-1_1p3_GO_CC.txt', sep='\t')
df_PCM1_MF = pd.read_csv('DAVID annotations/PCM-1/PCM-1_1p3_GO_MF.txt', sep='\t')
df_PCM1_KEGG = pd.read_csv('DAVID annotations/PCM-1/PCM-1_1p3_KEGG.txt', sep='\t')

df_PCM1_BP_simp = pullout_GO_info(df_PCM1_BP)
df_PCM1_CC_simp = pullout_GO_info(df_PCM1_CC)
df_PCM1_MF_simp = pullout_GO_info(df_PCM1_MF)
dfs_PCM1 = [df_PCM1_BP_simp, df_PCM1_CC_simp, df_PCM1_MF_simp]
df_PCM1_comb = pd.concat(dfs_PCM1)

df_PCM1_BP_simp.to_excel('Output/PCM-1/PCM-1_BP.xlsx')
df_PCM1_CC_simp.to_excel('Output/PCM-1/PCM-1_CC.xlsx')
df_PCM1_MF_simp.to_excel('Output/PCM-1/PCM-1_MF.xlsx')
df_PCM1_comb.to_excel('Output/PCM-1/PCM-1_GO_combine.xlsx')


df_PCM2_BP = pd.read_csv('DAVID annotations/PCM-2/PCM-2_1p3_GO_BP.txt', sep='\t')
df_PCM2_CC = pd.read_csv('DAVID annotations/PCM-2/PCM-2_1p3_GO_CC.txt', sep='\t')
df_PCM2_MF = pd.read_csv('DAVID annotations/PCM-2/PCM-2_1p3_GO_MF.txt', sep='\t')
df_PCM2_KEGG = pd.read_csv('DAVID annotations/PCM-2/PCM-2_1p3_KEGG.txt', sep='\t')

df_PCM2_BP_simp = pullout_GO_info(df_PCM2_BP)
df_PCM2_CC_simp = pullout_GO_info(df_PCM2_CC)
df_PCM2_MF_simp = pullout_GO_info(df_PCM2_MF)
dfs_PCM2 = [df_PCM2_BP_simp, df_PCM2_CC_simp, df_PCM2_MF_simp]
df_PCM2_comb = pd.concat(dfs_PCM2)

df_PCM2_BP_simp.to_excel('Output/PCM-2/PCM-2_BP.xlsx')
df_PCM2_CC_simp.to_excel('Output/PCM-2/PCM-2_CC.xlsx')
df_PCM2_MF_simp.to_excel('Output/PCM-2/PCM-2_MF.xlsx')
df_PCM2_comb.to_excel('Output/PCM-2/PCM-2_GO_combine.xlsx')

# 